In [1]:
'''
Author: Umar Khan
Date: 12/31/2024

App Description: A GUI interface to load, transform and perform statistical analysis on student data.

'''

# Import necessary packages
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# ----------------------------------------------------------------------------
# DataFrame Globals (optional, not strictly required)
# ----------------------------------------------------------------------------
df_user = None
df_activity = None
df_component_codes = None
df_removed = None
df_renamed_user = None
df_renamed_activity = None
df_merged = None
df_reshaped = None
df_counted = None  # after COUNT

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Data Cleaning & Analysis Prototype")

        # Make the window full-screen on Windows.
        # For other OS, you could do: self.attributes("-fullscreen", True)
        self.state("zoomed")

        # ----------------------------------------------------------------
        # STYLING WITH ttk.Style
        # ----------------------------------------------------------------
        style = ttk.Style()
        style.theme_use("clam")

        bg_color = "#f0f0f0"
        btn_bg = "#006699"
        highlight_color = "#cce6ff"

        self.configure(bg=bg_color)

        # Frame style
        style.configure("TFrame", background=bg_color)

        # Label style
        style.configure("TLabel", background=bg_color, foreground="black", font=("Arial", 11))

        # Button style
        style.configure("Large.TButton",
                        foreground="white",
                        background=btn_bg,
                        font=("Arial", 10, "bold"),
                        padding=5)
        style.map("Large.TButton",
                  background=[("active", "#005580")])

        # Combobox style
        style.configure("TCombobox", background="white", foreground="black", font=("Arial", 10), padding=3)

        # Treeview style
        style.configure("Treeview",
                        font=("Arial", 10),
                        background="white",
                        foreground="black",
                        rowheight=22,
                        fieldbackground="white")
        style.configure("Treeview.Heading",
                        font=("Arial", 10, "bold"),
                        background=highlight_color,
                        foreground="black")

        # Radio/Check button styling
        style.configure("Large.TRadiobutton",
                        background=bg_color,
                        font=("Arial", 10),
                        padding=3)
        style.configure("Large.TCheckbutton",
                        background=bg_color,
                        font=("Arial", 10),
                        padding=3)

        # We include "COUNTED" so the user can preview that dataset too
        self.dataframes = {
            "USER_LOG": None,
            "ACTIVITY_LOG": None,
            "COMPONENT_CODES": None,
            "REMOVED": None,
            "RENAMED_ACTIVITY": None,
            "MERGED": None,
            "RESHAPED": None,
            "COUNTED": None
        }

        # Status bar at the bottom
        self.status_var = tk.StringVar(value="Welcome!")
        self.status_bar = ttk.Label(self, textvariable=self.status_var, anchor="w", relief="sunken")
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)

        # Left Frame for step buttons
        self.left_frame = ttk.Frame(self, padding=10)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y)

        # Build left panel
        self.build_left_panel()

        # Build right panel (PanedWindow with top data preview + bottom stats/plot)
        self.build_right_panel()

        self.set_status("Application started.")

    # ----------------------------------------------------------------
    # Utility: Set status bar at the bottom of APP
    # ----------------------------------------------------------------
    def set_status(self, message: str):
        self.status_var.set(message)
        self.update_idletasks()

    # ----------------------------------------------------------------
    # Build Left Panel 
    # - Adding buttons
    # ----------------------------------------------------------------
    def build_left_panel(self):
        # Keep button width constant
        btn_width = 18

        # 1) Load CSVs Button
        self.btn_load_csv = ttk.Button(
            self.left_frame, text="Load CSVs",
            style="Large.TButton", command=self.load_csvs,
            width=btn_width
        )
        self.btn_load_csv.pack(pady=5, anchor="w")

        # 2) REMOVE Button
        self.btn_remove = ttk.Button(
            self.left_frame, text="REMOVE",
            style="Large.TButton", command=self.remove_system_folder,
            width=btn_width
        )
        self.btn_remove.pack(pady=5, anchor="w")

        # 3) RENAME Button
        self.btn_rename = ttk.Button(
            self.left_frame, text="RENAME",
            style="Large.TButton", command=self.rename_columns,
            width=btn_width
        )
        self.btn_rename.pack(pady=5, anchor="w")

        # 4) MERGE Button
        self.btn_merge = ttk.Button(
            self.left_frame, text="MERGE",
            style="Large.TButton", command=self.merge_dfs,
            width=btn_width
        )
        self.btn_merge.pack(pady=5, anchor="w")

        # 5) RESHAPE Button
        self.btn_reshape = ttk.Button(
            self.left_frame, text="RESHAPE",
            style="Large.TButton", command=self.reshape_data,
            width=btn_width
        )
        self.btn_reshape.pack(pady=5, anchor="w")

        # 6) COUNT Button
        self.btn_count = ttk.Button(
            self.left_frame, text="COUNT",
            style="Large.TButton", command=self.count_interactions,
            width=btn_width
        )
        self.btn_count.pack(pady=5, anchor="w")

        # Save/Load JSON Button
        self.btn_save_json = ttk.Button(
            self.left_frame, text="Save to JSON",
            style="Large.TButton", command=self.save_to_json,
            width=btn_width
        )
        self.btn_save_json.pack(pady=5, anchor="w")

        self.btn_load_json = ttk.Button(
            self.left_frame, text="Load from JSON",
            style="Large.TButton", command=self.load_from_json,
            width=btn_width
        )
        self.btn_load_json.pack(pady=5, anchor="w")

        # Stats Button
        stats_label = ttk.Label(self.left_frame, text="Component for Stats:")
        stats_label.pack(pady=(20, 0), anchor="w")

        self.stats_var = tk.StringVar(value="Quiz")
        stats_components = ["Quiz", "Lecture", "Assignment", "Attendence", "Survey"]
        for comp in stats_components:
            rb = ttk.Radiobutton(self.left_frame, text=comp,
                                 variable=self.stats_var, value=comp,
                                 style="Large.TRadiobutton")
            rb.pack(anchor="w")

        self.btn_stats = ttk.Button(
            self.left_frame, text="Output Statistics",
            style="Large.TButton", command=self.output_statistics,
            width=btn_width
        )
        self.btn_stats.pack(pady=5, anchor="w")

        # Correlation Button
        corr_label = ttk.Label(self.left_frame, text="Components for Correlation:")
        corr_label.pack(pady=(20, 0), anchor="w")

        self.corr_options = ["Assignment", "Quiz", "Lecture", "Book", "Project", "Course"]
        self.corr_vars = {}
        for c in self.corr_options:
            var = tk.BooleanVar(value=True)
            chk = ttk.Checkbutton(self.left_frame, text=c,
                                  variable=var, style="Large.TCheckbutton")
            chk.pack(anchor="w")
            self.corr_vars[c] = var

        self.btn_corr = ttk.Button(
            self.left_frame, text="Output Correlation",
            style="Large.TButton", command=self.output_correlation,
            width=btn_width
        )
        self.btn_corr.pack(pady=5, anchor="w")

    # ----------------------------------------------------------------
    # Build Right Panel (Data Preview + Stats/Plot)
    # ----------------------------------------------------------------
    def build_right_panel(self):
        self.right_paned = ttk.Panedwindow(self, orient=tk.VERTICAL)
        self.right_paned.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        # Top: data preview
        self.data_preview_frame = ttk.Frame(self.right_paned, padding=5)

        # Bottom: stats/plot
        self.bottom_frame = ttk.Frame(self.right_paned, padding=5)

        self.right_paned.add(self.data_preview_frame, weight=1)
        self.right_paned.add(self.bottom_frame, weight=1)

        # 1) Combobox near the data preview
        combo_container = ttk.Frame(self.data_preview_frame)
        combo_container.pack(side=tk.TOP, fill=tk.X)

        dataset_label = ttk.Label(combo_container, text="Select Dataset to Preview:")
        dataset_label.pack(side=tk.LEFT, padx=5)

        self.data_var = tk.StringVar()
        dataset_names = list(self.dataframes.keys())
        self.data_var.set(dataset_names[0])
        self.data_selector = ttk.Combobox(combo_container,
                                          textvariable=self.data_var,
                                          values=dataset_names,
                                          state="readonly",
                                          width=25)
        self.data_selector.pack(side=tk.LEFT, padx=5)
        self.data_selector.bind("<<ComboboxSelected>>", self.on_data_select)

        preview_label = ttk.Label(self.data_preview_frame, text="Data Preview (Top 5 Rows):")
        preview_label.pack(anchor="nw")

        # 2) The Treeview + Scrollbars
        self.tree_container = ttk.Frame(self.data_preview_frame)
        self.tree_container.pack(fill=tk.BOTH, expand=True)

        self.data_tree = ttk.Treeview(self.tree_container, show="headings")
        self.data_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Vertical scrollbar
        self.data_tree_scrollbar_y = ttk.Scrollbar(self.tree_container, orient="vertical",
                                                   command=self.data_tree.yview)
        self.data_tree_scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

        # Horizontal scrollbar
        self.data_tree_scrollbar_x = ttk.Scrollbar(self.data_preview_frame, orient="horizontal",
                                                   command=self.data_tree.xview)
        self.data_tree_scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)

        self.data_tree.configure(yscrollcommand=self.data_tree_scrollbar_y.set,
                                 xscrollcommand=self.data_tree_scrollbar_x.set)

        # Bottom: Stats + Plot
        self.stats_frame = ttk.Frame(self.bottom_frame)
        self.stats_frame.pack(side=tk.TOP, fill=tk.X, expand=False)

        stats_output_label = ttk.Label(self.stats_frame, text="Statistical Outputs / Messages:")
        stats_output_label.pack(anchor="nw")

        self.stats_text = tk.Text(self.stats_frame, wrap="none", height=8)
        self.stats_text.pack(fill=tk.X, padx=5, pady=5)

        self.plot_frame = ttk.Frame(self.bottom_frame)
        self.plot_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    # ----------------------------------------------------------------
    # When user picks a dataset in the Combobox
    # ----------------------------------------------------------------
    def on_data_select(self, event):
        selected_key = self.data_var.get()
        df = self.dataframes.get(selected_key, None)
        self.display_df_in_treeview(df)
        self.set_status(f"Previewing dataset: {selected_key}")

    # ----------------------------------------------------------------
    # Display top 5 rows with auto-sized columns
    # ----------------------------------------------------------------
    def display_df_in_treeview(self, df: pd.DataFrame):
        self.data_tree.delete(*self.data_tree.get_children())
        self.data_tree["columns"] = []

        if df is None or df.empty:
            return

        display_df = df.head(7).copy()
        columns = list(display_df.columns)
        self.data_tree["columns"] = columns

        # Create headings
        for col in columns:
            self.data_tree.heading(col, text=col)

        # Insert row data
        for _, row in display_df.iterrows():
            vals = list(row.values)
            self.data_tree.insert("", tk.END, values=vals)


    # ----------------------------------------------------------------
    def display_stats_message(self, msg: str):
        self.stats_text.delete("1.0", tk.END)
        self.stats_text.insert(tk.END, msg)

    # ----------------------------------------------------------------
    def clear_preview(self):
        self.data_tree.delete(*self.data_tree.get_children())
        self.data_tree["columns"] = []
        self.stats_text.delete("1.0", tk.END)

    # ----------------------------------------------------------------
    # LOAD CSVs (Remove duplicates), then write them to JSON,
    # then read them from JSON so we "keep them as JSON" in memory
    # ----------------------------------------------------------------
    def load_csvs(self):
        global df_user, df_activity, df_component_codes
        try:
            # 1) Read CSV
            df_user = pd.read_csv("USER_LOG.csv", date_format='%Y%m%d')
            df_activity = pd.read_csv("ACTIVITY_LOG.csv")
            df_component_codes = pd.read_csv("COMPONENT_CODES.csv")

            # 2) drop duplicates
            df_user.drop_duplicates(inplace=True)
            df_activity.drop_duplicates(inplace=True)
            df_component_codes.drop_duplicates(inplace=True)

            # 3) Write each out to JSON
            df_user.to_json("user_log.json", orient="records")
            df_activity.to_json("activity_log.json", orient="records")
            df_component_codes.to_json("component_codes.json", orient="records")

            # 4) Read them back from JSON into memory
            #    so that everything is effectively "kept" in JSON
            df_user = pd.read_json("user_log.json", orient="records")
            df_activity = pd.read_json("activity_log.json", orient="records")
            df_component_codes = pd.read_json("component_codes.json", orient="records")

            # Store in dataframes dictionary
            self.dataframes["USER_LOG"] = df_user
            self.dataframes["ACTIVITY_LOG"] = df_activity
            self.dataframes["COMPONENT_CODES"] = df_component_codes

            self.display_stats_message(
                "CSV files loaded & duplicates removed.\n"
                "Each DF saved to JSON (user_log.json, activity_log.json, component_codes.json)\n"
                "Then reloaded from JSON."
            )
            self.set_status("CSV→JSON→Loaded in memory.")

            # Show USER_LOG data immediately
            self.data_var.set("USER_LOG")
            self.display_df_in_treeview(df_user)
            self.set_status("Previewing dataset: USER_LOG")

            messagebox.showinfo("Success", "CSV files loaded, saved to JSON, and reloaded.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load CSV files:\n{e}")

    # ----------------------------------------------------------------
    # REMOVE: remove rows with 'System' or 'Folder' (from df_activity)
    # ----------------------------------------------------------------
    def remove_system_folder(self):
        global df_activity, df_removed
        if df_activity is None:
            messagebox.showwarning("Warning", "Please load CSVs first.")
            return

        try:
            df_removed = df_activity.copy()
            drop_index = df_removed[df_removed["Component"].isin(["System", "Folder"])].index
            df_removed.drop(drop_index, inplace=True)

            self.dataframes["REMOVED"] = df_removed

            self.display_stats_message("REMOVE step complete (System/Folder removed).")
            self.set_status("REMOVE step completed.")
            messagebox.showinfo("Success", "REMOVE step completed!")
        except Exception as e:
            messagebox.showerror("Error", f"REMOVE step failed:\n{e}")

    # ----------------------------------------------------------------
    # RENAME
    # ----------------------------------------------------------------
    def rename_columns(self):
        global df_user, df_removed, df_renamed_user, df_renamed_activity
        if df_user is None or df_removed is None:
            messagebox.showwarning("Warning", "Please load CSVs and do REMOVE step first.")
            return

        try:
            df_renamed_user = df_user.copy()
            if "User Full Name *Anonymized" in df_renamed_user.columns:
                df_renamed_user.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)

            df_renamed_activity = df_removed.copy()
            if "User Full Name *Anonymized" in df_renamed_activity.columns:
                df_renamed_activity.rename(columns={"User Full Name *Anonymized": "User_ID"}, inplace=True)

            self.dataframes["RENAMED_ACTIVITY"] = df_renamed_activity

            self.display_stats_message("RENAME step complete (User Full Name -> User_ID).")
            self.set_status("RENAME step completed.")
            messagebox.showinfo("Success", "RENAME step completed!")
        except Exception as e:
            messagebox.showerror("Error", f"RENAME step failed:\n{e}")

    # ----------------------------------------------------------------
    # MERGE
    # ----------------------------------------------------------------
    def merge_dfs(self):
        global df_renamed_user, df_renamed_activity, df_component_codes, df_merged
        
        if df_renamed_user is None or df_renamed_activity is None or df_component_codes is None:
            messagebox.showwarning("Warning","Do REMOVE, RENAME, and load CSV first.")
            return

        try:
            df_temp = pd.merge(df_renamed_activity, df_renamed_user, on="User_ID", how="left")
            df_temp = pd.merge(df_temp, df_component_codes, on="Component", how="left")

            # Parse "Date" in the format "DD/mm/yyyy 00:00:00"
            df_temp["Date"] = pd.to_datetime(
                df_temp["Date"], 
                format="%Y/%m/%d",
                errors="coerce"
            )

            # Drop rows with invalid or missing dates
            df_temp.dropna(subset=["Date"], inplace=True)

            # Extract month as "01","02","03", etc.
            df_temp["Month"] = df_temp["Date"].dt.strftime("%m")

            df_merged = df_temp
            self.dataframes["MERGED"] = df_merged

            self.display_stats_message("MERGE step complete (Date->Month).")
            self.set_status("MERGE done.")
            messagebox.showinfo("Success","MERGE step completed!")
        except Exception as e:
            messagebox.showerror("Error",f"MERGE step failed:\n{e}")

    # ----------------------------------------------------------------
    # RESHAPE
    # ----------------------------------------------------------------
    def reshape_data(self):
        global df_merged, df_reshaped
        
        if df_merged is None:
            messagebox.showwarning("Warning", "Please MERGE first.")
            return

        try:
            grouped = (
                df_merged.groupby(["User_ID","Month","Component"])
                         .size()
                         .reset_index(name="Interaction_Count")
            )
            df_pivot = grouped.pivot_table(
                index=["User_ID","Month"],
                columns="Component",
                values="Interaction_Count",
                fill_value=0
            )
            df_pivot.reset_index(inplace=True)

            df_reshaped = df_pivot
            self.dataframes["RESHAPED"] = df_reshaped

            self.display_stats_message("RESHAPE step complete (Pivoted data).")
            self.set_status("RESHAPE done.")
            messagebox.showinfo("Success", "RESHAPE step completed!")
        except Exception as e:
            messagebox.showerror("Error", f"RESHAPE step failed:\n{e}")

    # ----------------------------------------------------------------
    # COUNT
    # ----------------------------------------------------------------
    def count_interactions(self):
        global df_reshaped, df_counted
        if df_reshaped is None:
            messagebox.showwarning("Warning", "Please complete RESHAPE before COUNT.")
            return

        try:
            df_counted = df_reshaped.copy()
            component_cols = [c for c in df_counted.columns if c not in ["User_ID", "Month"]]
            df_counted["Total_Interactions"] = df_counted[component_cols].sum(axis=1)

            self.dataframes["COUNTED"] = df_counted

            self.display_stats_message("COUNT step complete (Total_Interactions added).")
            self.set_status("COUNT step completed.")
            messagebox.showinfo("Success", "COUNT step completed!")
        except Exception as e:
            messagebox.showerror("Error", f"COUNT step failed:\n{e}")

    # ----------------------------------------------------------------
    # Save to JSON (for the final df_reshaped)
    # ----------------------------------------------------------------
    def save_to_json(self):
        global df_reshaped
        if df_reshaped is None:
            messagebox.showwarning("Warning", "No reshaped data found. Please RESHAPE or LOAD from JSON first.")
            return

        try:
            file_path = filedialog.asksaveasfilename(
                defaultextension=".json",
                filetypes=[("JSON Files", "*.json"), ("All Files", "*.*")]
            )
            if not file_path:
                return

            df_reshaped.to_json(file_path, orient="records")
            self.display_stats_message(f"Data saved to JSON:\n{file_path}")
            self.set_status("Data saved.")
            messagebox.showinfo("Success", f"Data saved to:\n{file_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save JSON:\n{e}")

    # ----------------------------------------------------------------
    # Load from JSON (replaces df_reshaped)
    # ----------------------------------------------------------------
    def load_from_json(self):
        global df_reshaped
        try:
            file_path = filedialog.askopenfilename(
                filetypes=[("JSON Files", "*.json"), ("All Files", "*.*")]
            )
            if not file_path:
                return

            df_reshaped = pd.read_json(file_path, orient="records")
            self.dataframes["RESHAPED"] = df_reshaped

            self.display_stats_message(f"Data loaded from JSON:\n{file_path}")
            self.set_status("JSON loaded.")
            messagebox.showinfo("Success", f"Data loaded from:\n{file_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load JSON:\n{e}")

    # ----------------------------------------------------------------
    # Output Statistics (mean, median, mode) for selected comp
    # ----------------------------------------------------------------
    def output_statistics(self):
        global df_reshaped
        if df_reshaped is None:
            messagebox.showwarning("Warning","Please complete transformations or load JSON first.")
            return

        selected_comp = self.stats_var.get()
        try:
            if selected_comp not in df_reshaped.columns:
                messagebox.showwarning("Warning",f"Component '{selected_comp}' not found in pivoted data.")
                return

            # Clear old plot
            for widget in self.plot_frame.winfo_children():
                widget.destroy()

            # Subset for stats
            df_sel = df_reshaped[["User_ID","Month",selected_comp]].copy()
            df_sel.rename(columns={selected_comp:"Interaction_Count"}, inplace=True)

            monthly = df_sel.groupby("Month")["Interaction_Count"]
            monthly_mean = monthly.mean()
            monthly_median = monthly.median()

            def mode_func(x):
                m = x.mode()
                return m.iloc[0] if not m.empty else np.nan
            monthly_mode = monthly.apply(mode_func)

            overall_mean = df_sel["Interaction_Count"].mean()
            overall_median = df_sel["Interaction_Count"].median()
            overall_mode_series = df_sel["Interaction_Count"].mode()
            overall_mode = overall_mode_series.iloc[0] if not overall_mode_series.empty else np.nan

            # Build message
            msg = (
                f"STATISTICS FOR COMPONENT: {selected_comp}\n\n"
                "Monthly Mean & Median & Mode:\n"
                f"{pd.DataFrame({'Mean': monthly_mean, 'Median': monthly_median, 'Mode': monthly_mode}).to_string()}\n\n"
                f"Entire Semester:\n"
                f"  Mean:   {overall_mean:.2f}\n"
                f"  Median: {overall_median:.2f}\n"
                f"  Mode:   {overall_mode}\n"
            )
            self.display_stats_message(msg)

            # Plot mean, median, mode as bar charts
            import matplotlib.pyplot as plt

            fig, axes = plt.subplots(1, 3, figsize=(12,4))

            # Sort by index if needed
            monthly_mean = monthly_mean.sort_index()
            monthly_median = monthly_median.sort_index()
            monthly_mode = monthly_mode.sort_index()

            # Mean
            axes[0].bar(monthly_mean.index, monthly_mean.values, color="skyblue")
            axes[0].set_title("Monthly Mean")
            axes[0].set_xlabel("Month")
            axes[0].set_ylabel("Mean Interactions")
            axes[0].tick_params(axis='x', rotation=45)

            # Median
            axes[1].bar(monthly_median.index, monthly_median.values, color="salmon")
            axes[1].set_title("Monthly Median")
            axes[1].set_xlabel("Month")
            axes[1].set_ylabel("Median Interactions")
            axes[1].tick_params(axis='x', rotation=45)

            # Mode
            axes[2].bar(monthly_mode.index, monthly_mode.values, color="limegreen")
            axes[2].set_title("Monthly Mode")
            axes[2].set_xlabel("Month")
            axes[2].set_ylabel("Mode Interactions")
            axes[2].tick_params(axis='x', rotation=45)

            fig.tight_layout()

            canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

            self.set_status("Output Statistics completed.")
            messagebox.showinfo("Success","Output Statistics (with monthly bar charts) completed!")

        except Exception as e:
            messagebox.showerror("Error",f"Failed to compute statistics:\n{e}")

    # ----------------------------------------------------------------
    # Output Correlation (heatmap)
    # ----------------------------------------------------------------
    def output_correlation(self):
        global df_reshaped
        if df_reshaped is None:
            messagebox.showwarning("Warning", "Please complete transformations or load JSON first.")
            return

        selected_components = [c for c in self.corr_vars if self.corr_vars[c].get() is True]

        self.display_stats_message("")
        for widget in self.plot_frame.winfo_children():
            widget.destroy()

        if len(selected_components) < 2:
            messagebox.showinfo("Info", "Please select at least two components for correlation.")
            return

        try:
            missing_cols = [c for c in selected_components if c not in df_reshaped.columns]
            if missing_cols:
                messagebox.showwarning("Warning", f"Missing in data: {missing_cols}")
                return

            df_corr_base = df_reshaped[selected_components].copy()
            if df_corr_base.shape[1] <= 1:
                messagebox.showinfo("Info", "Not enough data to compute correlation.")
                return

            corr_matrix = df_corr_base.corr()

            fig, ax = plt.subplots(figsize=(6, 4))
            sns.heatmap(corr_matrix, annot=True, cmap="viridis", ax=ax)
            ax.set_title("Correlation among Selected Components")

            canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

            self.display_stats_message("Correlation heatmap generated!\n\n" + corr_matrix.to_string())
            self.set_status("Correlation done.")
            messagebox.showinfo("Success", "Correlation heatmap generated!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to output correlation:\n{e}")


# ----------------------------------------------------------------------------
# MAIN: Run
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    app = App()
    app.mainloop()
